In [1]:
!pip install gensim
!pip install SudachiPy==0.5.4 ginza==4.0.6 ja-ginza==4.0.0 
!pip install pytorch_lightning
!pip install fugashi
!pip install ipadic

You should consider upgrading via the '/opt/venv/bin/python3 -m pip install --upgrade pip' command.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 KB 174.6 kB/s eta 0:00:00 0:00:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 MB 8.5 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 7.8 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 KB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.5 MB/s eta 0:00:00
  Created wheel for SudachiPy: filename=SudachiPy-0.5.4-cp39-cp39-linux_x86_64.whl size=1295759 sha256=d323ab5a7551e363cc58ed852312e1f3160c6f077117296618d857522c848fd9
  Stored in directory: /root/.cache/pip/wheels/f4/21/92/d3c914af0227e3faf483523a4e72537236c928459a

### データ読み込み

In [43]:
import pandas as pd
df = pd.read_excel("Machine_Learning_Data.xlsx")
columns = df.columns[1:3]
df1 = df[columns]
df1.columns = ['分類', 'テキスト']

df_txt = df1["テキスト"]
df_cl = df1["分類"]

# 混合ラベル => ネガティブ化
for i in range(len(df_cl)):
    if "\u3000" in df.at[i, "サウンド バイトのテキスト"]:
        v = df.at[i, "サウンド バイトのテキスト"]
        df.at[i, "サウンド バイトのテキスト"] = v.replace("\u3000", "")
        
    if df1.at[i,"分類"] == "混合":
        df1.at[i,"分類"] = "ネガティブ"
        

print("Text_length:",len(df_txt), "Classification_length:",len(df_cl))
df["サウンド バイトのテキスト"]

Text_length: 3487 Classification_length: 3487


0       【ミニカー予約速報】EIDOLON (アイドロン) 1/43 トヨタ GR86 RZ 202...
1                         トヨタ ライズの買取価格は？査定額や相場がいくらか調査した！ 
2       やっぱりレクサスはトヨタと違う！ 新型ESにあってクラウンにはない高級感とは？(GQ JAP...
3                             【朗報】トヨタの新型プリウス、ガチのマジでかっこいい 
4                 将来、2輪は水素直噴カワサキH2SX、4輪はトヨタ水素直噴カローラで水素生活か
                              ...                        
3482                            【画像】トヨタさん、新型プリウスがヤバイと話題へ 
3483                  トヨタ、新型「プリウス」ティザー公開！ 11月16日ワールドプレミア 
3484    世界中の自動車メーカーの社長が自家用車に欲しいと思ってる。それがプリウスなんだよね。 トヨタ...
3485                       トヨタ新型「プリウス」世界初公開! PHEVは223馬力  
3486            【朗報】トヨタの新型プリウス、外見が話題になっている理由がコレｗｗｗｗｗｗｗｗｗ 
Name: サウンド バイトのテキスト, Length: 3487, dtype: object

### 分類ベクトル化 (one-hot)

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer

#TF-IDF
tfidf_vectorizer = TfidfVectorizer() #token_pattern="(?u)\b\w\w+\b"
tfidf = tfidf_vectorizer.fit_transform(df_cl)

df2 = pd.DataFrame(
    data = tfidf.toarray(),
    columns = tfidf_vectorizer.get_feature_names()
)
df2

/opt/venv/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,ニュートラル,ネガティブ,ポジティブ
0,1.0,0.0,0.0
1,1.0,0.0,0.0
2,0.0,0.0,1.0
3,0.0,0.0,1.0
4,1.0,0.0,0.0
...,...,...,...
3482,0.0,0.0,1.0
3483,1.0,0.0,0.0
3484,0.0,0.0,1.0
3485,0.0,0.0,1.0


### テキストデータの加工

In [45]:
texts = []
classes = []
for txt, cl in zip(df_txt, df_cl):
    #語尾の空白削除
    if txt[-1] == " " :
        txt = txt[:-1]
        if txt[-1] == " ":
            txt = txt[:-1]
    texts.append(txt)
    classes.append(cl)

In [46]:
# データの結合化
df = pd.concat([df1, df2], axis=1)
df.columns = ['Class', 'Text', 'Cl_Neutral', 'Cl_Negative', 'Cl_Positive']
df.head()

,Class,Text,Cl_Neutral,Cl_Negative,Cl_Positive
0,ニュートラル,【ミニカー予約速報】EIDOLON (アイドロン) 1/43 トヨタ GR86 RZ 202...,1.0,0.0,0.0
1,ニュートラル,トヨタ ライズの買取価格は？査定額や相場がいくらか調査した！,1.0,0.0,0.0
2,ポジティブ,やっぱりレクサスはトヨタと違う！ 新型ESにあってクラウンにはない高級感とは？(GQ JAP...,0.0,0.0,1.0
3,ポジティブ,【朗報】トヨタの新型プリウス、ガチのマジでかっこいい,0.0,0.0,1.0
4,ニュートラル,将来、2輪は水素直噴カワサキH2SX、4輪はトヨタ水素直噴カローラで水素生活か,1.0,0.0,0.0


### テキスト分かち書き

In [ ]:
import MeCab
wakati_text = []
wakati_changes_text = []
tagger =  MeCab.Tagger(r'-Owakati') #-Fs%f[6] -Us%m

for line in texts:
    res = tagger.parse(line)
    results = res.split(" ")[:-1]
    print(results)
    # if "\u3000" in results:
        # results = results.replace("\u3000", "")
    with open("wakati_changes.txt", "a", encoding="utf-8") as of:
                of.write(res[1:])
    wakati_changes_text.append(res)
    wakati_text.append(results)
print("Finish")


['【', 'ミニカー', '予約', '速報', '】', 'EIDOLON', '(', 'アイ', 'ドロン', ')', '1', '/', '43', 'トヨタ', 'GR', '86', 'RZ', '2021', 'ブライト', 'ブルー', 'の', '予約', 'を', '開始', 'し', 'まし', 'た', '！', '#', 'ガリバー', 'ミニカー', '#', 'ガリバー', 'ミニカー', '予約', '#', 'ミニカー', '#', 'minicar', '#', 'modelcar', '#', 'EIDOLON', '#', 'アイドロン', '#', 'トヨタ', '#', 'TOYOTA', '#', 'GR', '86']
['トヨタ', 'ライズ', 'の', '買取', '価格', 'は', '？', '査定', '額', 'や', '相場', 'が', 'いくら', 'か', '調査', 'し', 'た', '！']
['やっぱり', 'レクサス', 'は', 'トヨタ', 'と', '違う', '！', '新型', 'ES', 'に', 'あっ', 'て', 'クラウン', 'に', 'は', 'ない', '高級', '感', 'と', 'は', '？', '(', 'GQ', 'JAPAN', ')']
['【', '朗報', '】', 'トヨタ', 'の', '新型', 'プリウス', '、', 'ガ', 'チ', 'の', 'マジ', 'で', 'かっこ', 'いい']
['将来', '、', '2', '輪', 'は', '水素', '直噴', 'カワサキ', 'H', '2', 'SX', '、', '4', '輪', 'は', 'トヨタ', '水素', '直噴', 'カローラ', 'で', '水素', '生活', 'か']
['【', '画像', '】', 'トヨタ', 'の', '新型', 'プリウス', '、', 'ガ', 'チ', 'の', 'マジ', 'で', 'かっこ', 'いい']
['【', 'トヨタ', 'プリウス', '新型', '】', 'パワー', 'プラント', 'は', '3', '車', '類', '、', 'PHEV', 'は', 'HV', 'より', '強力', '

### テキストのベクトル化

In [49]:
from gensim.models import word2vec
docs = word2vec.LineSentence("wakati_changes.txt")
model = word2vec.Word2Vec(
    docs,
    vector_size=1,
    min_count=1,
    window=5,
    epochs=30
)
model.save("to_toyota_innetbase.model")
print("text vector modeling success")

text vector modeling success


In [50]:
from gensim.models import word2vec
model   = word2vec.Word2Vec.load("to_toyota_innetbase.model")

vector_changes = []
for txt in wakati_text:
    for i in txt:
        results = model.wv[i]
        for vector in results:
            vector_changes.append(vector)

KeyError: "Key '\u3000' not present"

In [42]:
[df["サウンド バイトのテキスト"]=="\u3000"]



[0       False
 1       False
 2       False
 3       False
 4       False
         ...  
 3482    False
 3483    False
 3484    False
 3485    False
 3486    False
 Name: サウンド バイトのテキスト, Length: 3487, dtype: bool]

In [15]:
wakati_changes_text

['【 ミニカー 予約 速報 】 EIDOLON ( アイ ドロン ) 1 / 43 トヨタ GR 86 RZ 2021 ブライト ブルー の 予約 を 開始 し まし た ！ # ガリバー ミニカー # ガリバー ミニカー 予約 # ミニカー # minicar # modelcar # EIDOLON # アイドロン # トヨタ # TOYOTA # GR 86 \n',
 'トヨタ ライズ の 買取 価格 は ？ 査定 額 や 相場 が いくら か 調査 し た ！ \n',
 'やっぱり レクサス は トヨタ と 違う ！ 新型 ES に あっ て クラウン に は ない 高級 感 と は ？ ( GQ JAPAN ) \n',
 '【 朗報 】 【 うれしい 】 【 画像 】 トヨタ の 新型 プリウス 、 ガ チ の マジ で かっこ いい \n',
 '将来 、 2 輪 は 水素 直噴 カワサキ H 2 SX 、 4 輪 は トヨタ 水素 直噴 カローラ で 水素 生活 か \n',
 '【 画像 】 トヨタ の 新型 プリウス 、 ガ チ の マジ で かっこ いい \n',
 '【 トヨタ プリウス 新型 】 パワー プラント は 3 車 類 、 PHEV は HV より 強力 ( レスポンス ) \n',
 '新しい プリウス は 、 トヨタ に 必要 な クルマ で ある 2022 年 11 月 16 日 に 発表 さ れ た 新型 「 プリウス 」 は おもしろい 。 と いう の は 、 「 ラブ か コモディティ か 」 を 、 トヨタ 社内 で 論議 し ながら 開発 し た ん だ そう だ 。 \n',
 '# Toyota カローラ ツーリング は 家族 や 荷物 が 増え たら 購入 を 検討 し たい 。 日本 車 で ステーション ワゴン は あまり 種類 ないし 、 何 より この デザイン 好き ? ? 立派 な デイ タイム ランニング ライト が すべて 方向 指示 器 に 変色 する ところ も 良い 。 価格 は 高 過ぎ ず 良心 的 。 \n',
 'プリウス を 威信 材 に する の が トヨタ は 上手かっ た ん です 。 \n',
 '次期 「 スー プラ 」 ? トヨタ が スポーツ クーペ 「 

Index(['Class', 'Text', 'Cl_Neutral', 'Cl_Negative', 'Cl_Positive'], dtype='object')

### 文章のベクトル化

In [42]:
from sklearn.feature_extraction.text import CountVectorizer
tagger =  MeCab.Tagger(r'-Owakati') 
for i, line in enumerate(texts):
    result = tagger.parse(line)
    wakati_changes_text.append(result)

count = CountVectorizer()
docs = np.array(wakati_changes_text)
x = count.fit_transform(docs)

### 分類データセットの作成

In [20]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline

import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.callbacks import EarlyStopping 

from sklearn import datasets
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2])